In [2]:
!nvidia-smi

Mon Sep 16 01:31:32 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [4]:
# Pip install method (recommended)

!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()
from ultralytics import YOLO

from IPython.display import display, Image
!pip install roboflow

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.8/112.6 GB disk)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.8
    Uninstalling idna-3.8:
      Successfully uninstalled idna-3.8


In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="Ldma71sWE7fMg8IJzdNA")
project = rf.workspace("amazonml").project("ehhehe-aek02")
version = project.version(6)
dataset = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to ehhehe-6 in yolov8:: 100%|██████████| 14836/14836 [00:02<00:00, 7131.33it/s]


In [ ]:
%cd {HOME}

!yolo task=detect mode=train model=yolov8s.pt data={dataset.location}/data.yaml epochs=25 imgsz=800 plots=True

/content
100% 21.5M/21.5M [00:00<00:00, 32.6MB/s]
/usr/local/lib/python3.10/dist-packages/ultralytics/nn/tasks.py:567: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return t

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install supervision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.5/151.5 kB 6.9 MB/s eta 0:00:00


In [6]:
import supervision as sv
import cv2
!pip install roboflow

In [7]:
!pip install pytesseract
!apt-get install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (5,574 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 123597 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [ ]:
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO
import pytesseract
import pandas as pd
import requests
from io import BytesIO
from PIL import Image

class ObjectDetectionTextExtractionPipeline:
    def __init__(self, model_path, tesseract_cmd_path):
        # Initialize the YOLO model
        self.model = YOLO(model_path)
        # Set up PyTesseract path
        pytesseract.pytesseract.tesseract_cmd = tesseract_cmd_path

    def download_image(self, image_link):
        """
        Download the image from a URL.
        """
        response = requests.get(image_link)
        img = Image.open(BytesIO(response.content))
        return cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)

    def detect_objects(self, image, threshold=0.01):
        """
        Run YOLO object detection on the image and return results with boxes filtered by threshold.
        """
        results = self.model(source=image, save=False)
        image = results[0].orig_img
        # Get the bounding boxes, confidences, and class IDs
        boxes = results[0].boxes.xyxy.cpu().numpy()
        confidences = results[0].boxes.conf.cpu().numpy()
        print("Conf: ", len(confidences))
        class_ids = results[0].boxes.cls.cpu().numpy().astype(int)
        print("Class ids: ", class_ids)
        # Filter boxes based on the confidence threshold
        filtered_indices = confidences > threshold
        boxes = boxes[filtered_indices]
        confidences = confidences[filtered_indices]
        class_ids = class_ids[filtered_indices]

        return image, boxes, confidences, class_ids, results[0].names

    def extract_text_from_box(self, image, box):
        """
        Extract text from the cropped bounding box region using PyTesseract.
        """
        x1, y1, x2, y2 = map(int, box)  # Extract coordinates
        cropped_img = image[y1:y2, x1:x2]

        # Convert to grayscale for better OCR performance
        gray_cropped_img = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2GRAY)

        # Use pytesseract to extract text
        extracted_text = pytesseract.image_to_string(gray_cropped_img)
        return extracted_text

    def run_pipeline_on_row(self, row, threshold=0.01):
        """
        Process one row of the dataframe.
        """
        image = self.download_image(row['image_link'])
        entity_name = row['entity_name']
        # Detect objects in the image
        image, boxes, confidences, class_ids, class_names = self.detect_objects(image, threshold)

        # Filter for the class matching the entity_name
        print("Class name: ", class_names)
        matched_indices = [i for i, class_id in enumerate(class_ids) if class_names[class_id] == entity_name]

        if len(matched_indices) == 0:
            print("No matching indices")
            return None  # No matching object found

        # Choose the one with the highest confidence
        best_index = max(matched_indices, key=lambda i: confidences[i])
        best_box = boxes[best_index]

        # Extract text from the chosen bounding box
        extracted_text = self.extract_text_from_box(image, best_box)

        return extracted_text

    def run_pipeline(self, df, threshold=0.01):
        """
        Run the pipeline on the entire dataframe.
        """
        output = []
        for _, row in df.iterrows():
            prediction = self.run_pipeline_on_row(row, threshold)
            output.append({'index': row['index'], 'prediction': prediction})
        return pd.DataFrame(output)


In [8]:
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO
import pytesseract
import pandas as pd
import requests
from io import BytesIO
from PIL import Image
import re

class ObjectDetectionTextExtractionPipeline:


    def __init__(self, model_path, tesseract_cmd_path):
        # Initialize the YOLO model
        self.model = YOLO(model_path)
        # Set up PyTesseract path
        pytesseract.pytesseract.tesseract_cmd = tesseract_cmd_path

        self.unit_normalization_map = {
        'lbs': 'pound', 'lb': 'pound', 'pounds': 'pound', 'pound': 'pound',
        'kg': 'kilogram', 'kgs': 'kilogram', 'kilograms': 'kilogram', 'kilogram': 'kilogram',
        'g': 'gram', 'grams': 'gram', 'gram': 'gram',
        'mg': 'milligram', 'milligrams': 'milligram', 'milligram': 'milligram',
        'oz': 'ounce', 'ounces': 'ounce', 'ounce': 'ounce',
        'ton': 'ton', 'tons': 'ton',
        'cm': 'centimetre', 'centimetres': 'centimetre', 'centimeter': 'centimetre', 'centimetres': 'centimetre',
        'mm': 'millimetre', 'millimeters': 'millimetre', 'millimetre': 'millimetre',
        'm': 'metre', 'meters': 'metre', 'metres': 'metre',
        'ft': 'foot', 'feet': 'foot', 'foot': 'foot',
        'yard': 'yard', 'yards': 'yard',
        'kv': 'kilovolt', 'kilovolts': 'kilovolt', 'kilovolt': 'kilovolt',
        'v': 'volt', 'volts': 'volt', 'volt': 'volt',
        'w': 'watt', 'watts': 'watt', 'watt': 'watt',
        'kw': 'kilowatt', 'kilowatts': 'kilowatt', 'kilowatt': 'kilowatt',
        'ml': 'millilitre', 'milliliters': 'millilitre', 'millilitres': 'millilitre',
        'litres': 'litre', 'litre': 'litre', 'l': 'litre',
        'gallons': 'gallon', 'gallon': 'gallon',
        'pints': 'pint', 'pint': 'pint',
        'quarts': 'quart', 'quart': 'quart',
        'cups': 'cup', 'cup': 'cup',
        'fluid ounces': 'fluid ounce', 'fluid ounce': 'fluid ounce',
        'cubic foot': 'cubic foot', 'cubic feet': 'cubic foot',
        'cubic inches': 'cubic inch', 'cubic inch': 'cubic inch',
        'cl': 'centilitre', 'centilitres': 'centilitre', 'centilitre': 'centilitre',
        'dl': 'decilitre', 'deciliters': 'decilitre', 'decilitre': 'decilitre',
        'imperial gallons': 'imperial gallon', 'imperial gallon': 'imperial gallon'
        }

    def download_image(self, image_link):
        """
        Download the image from a URL.
        """
        response = requests.get(image_link)
        img = Image.open(BytesIO(response.content))
        return cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)

    def detect_objects(self, image, threshold=0.01):
        """
        Run YOLO object detection on the image and return results with boxes filtered by threshold.
        """
        results = self.model(source=image, save=False, verbose=False)
        image = results[0].orig_img
        # Get the bounding boxes, confidences, and class IDs
        boxes = results[0].boxes.xyxy.cpu().numpy()
        confidences = results[0].boxes.conf.cpu().numpy()
        # print("Conf: ", len(confidences))
        class_ids = results[0].boxes.cls.cpu().numpy().astype(int)
        # print("Class ids: ", class_ids)
        # Filter boxes based on the confidence threshold
        filtered_indices = confidences > threshold
        boxes = boxes[filtered_indices]
        confidences = confidences[filtered_indices]
        class_ids = class_ids[filtered_indices]

        return image, boxes, confidences, class_ids, results[0].names

    def extract_text_from_box(self, image, box):
        """
        Extract text from the cropped bounding box region using PyTesseract.
        """
        x1, y1, x2, y2 = map(int, box)  # Extract coordinates
        cropped_img = image[y1:y2, x1:x2]

        # Convert to grayscale for better OCR performance
        gray_cropped_img = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2GRAY)

        # Use pytesseract to extract text
        extracted_text = pytesseract.image_to_string(gray_cropped_img)
        return extracted_text

    def parse_into_valid_form(self, entity_value):
        # Normalize and clean the input string
        entity_value = entity_value.lower().strip()

        # Extract the number using regex
        number_match = re.search(r'[-+]?\d*\.?\d+', entity_value)
        if not number_match:
            return ""

        number = number_match.group(0)

        # Initialize unit to None
        unit = None

        # Check for apostrophes (' for feet, " for inches)
        if "'" in entity_value:
            unit = 'foot'
        elif '"' in entity_value:
            unit = 'inch'
        else:
            # Find and normalize the unit from the normalization map
            for unit_variation, normalized_unit in self.unit_normalization_map.items():
                if unit_variation in entity_value:
                    unit = normalized_unit
                    break

        if unit is None:
            return ""  # If no valid unit is found

        # Convert the number to a float
        try:
            number = float(number)
        except ValueError:
            return ""

        # Return the cleaned number and unit in valid form
        return f"{number} {unit}"


    def run_pipeline_on_row(self, row, threshold=0.01):
      """
      Process one row of the dataframe with entity-specific rules.
      """
      image = self.download_image(row['image_link'])
      entity_name = row['entity_name']

      # Detect objects in the image
      image, boxes, confidences, class_ids, class_names = self.detect_objects(image, threshold)

      # Filter based on the entity_name
      if entity_name in ['width', 'height', 'depth']:
          # Any solution of class 'width', 'height', or 'depth' is viable
          matched_indices = [i for i, class_id in enumerate(class_ids) if class_names[class_id] in ['width', 'height', 'depth']]

      elif entity_name in ['item_weight', 'maximum_weight_recommendation']:
          # Any solution of class 'item_weight' or 'maximum_weight_recommendation' is viable
          matched_indices = [i for i, class_id in enumerate(class_ids) if class_names[class_id] in ['item_weight', 'maximum_weight_recommendation']]

      else:
          # For other entity names, match with the exact class name
          matched_indices = [i for i, class_id in enumerate(class_ids) if class_names[class_id] == entity_name]

      # If no matching object found, return an empty string
      if len(matched_indices) == 0:
          print("No matching indices")
          return ""

      # Choose the bounding box with the highest confidence
      best_index = max(matched_indices, key=lambda i: confidences[i])
      best_box = boxes[best_index]

      # Extract text from the chosen bounding box
      extracted_text = self.extract_text_from_box(image, best_box)

      return extracted_text


    def run_pipeline(self, df, threshold=0.01):
        """
        Run the pipeline on the entire dataframe.
        """
        output = []
        count=0
        for _, row in df.iterrows():
            count +=1
            print("Done: ", count)
            prediction = self.run_pipeline_on_row(row, threshold)
            output.append({'index': row['index'], 'prediction': prediction})
        return pd.DataFrame(output)

In [9]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/test.csv')

In [ ]:
df.head()

,index,image_link,group_id,entity_name
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth


In [11]:
import pandas as pd

def process_in_batches(pipeline, df, output_csv, batch_size=1000, threshold=0.01, start_index=38000):
    """
    Run the pipeline on the dataframe starting from the specified index in batches and append each batch to a CSV file.

    Parameters:
    pipeline : ObjectDetectionTextExtractionPipeline
        The initialized pipeline object.
    df : pandas.DataFrame
        The dataframe containing the rows to process.
    output_csv : str
        Path to the output CSV file where results will be saved.
    batch_size : int
        The number of rows to process per batch.
    threshold : float
        The threshold to use for object detection.
    start_index : int
        The starting index for processing rows.
    """
    total_rows = len(df)

    # Loop over the dataframe in batches starting from the specified index
    for start in range(start_index, total_rows, batch_size):
        end = min(start + batch_size, total_rows)
        print(f"Processing rows {start} to {end}")

        # Extract the current batch
        batch_df = df.iloc[start:end]

        # Update the index of the batch to start from the given start index
        batch_df.index = range(start, end)

        # Run the pipeline on the current batch
        batch_result = pipeline.run_pipeline(batch_df, threshold)

        # Append the batch results to the CSV file
        if start == start_index:
            # Write header only for the first batch
            batch_result.to_csv(output_csv, mode='w', header=True, index=True)
        else:
            # For subsequent batches, append without header
            batch_result.to_csv(output_csv, mode='a', header=False, index=True)

In [ ]:
if __name__ == "__main__":
    # Initialize your pipeline
    model_path = "/content/drive/MyDrive/best.pt"
    tesseract_cmd_path = r'/usr/bin/tesseract'
    pipeline = ObjectDetectionTextExtractionPipeline(model_path, tesseract_cmd_path)

    # Load your dataframe (example)
    df = pd.read_csv('/content/drive/MyDrive/test.csv')

    # Define the output CSV file path
    output_csv = "/content/drive/MyDrive/output_4.csv"

    # Process the dataframe in batches and directly append to CSV
    process_in_batches(pipeline, df, output_csv, batch_size=1000, threshold=0.01)

Processing rows 38000 to 39000
Done:  1


WARNING ⚠️ NMS time limit 0.550s exceeded


Streaming output truncated to the last 5000 lines.
Done:  330
No matching indices
Done:  331
Done:  332
Done:  333
Done:  334
Done:  335
Done:  336
Done:  337
Done:  338
Done:  339
Done:  340
Done:  341
Done:  342
Done:  343
Done:  344
Done:  345
No matching indices
Done:  346
No matching indices
Done:  347
Done:  348
Done:  349
No matching indices
Done:  350
Done:  351
Done:  352
No matching indices
Done:  353
Done:  354
Done:  355
Done:  356
Done:  357
Done:  358
Done:  359
Done:  360
Done:  361
Done:  362
Done:  363
Done:  364
Done:  365
Done:  366
Done:  367
Done:  368
Done:  369
No matching indices
Done:  370
No matching indices
Done:  371
Done:  372
Done:  373
Done:  374
Done:  375
No matching indices
Done:  376
No matching indices
Done:  377
No matching indices
Done:  378
No matching indices
Done:  379
Done:  380
Done:  381
Done:  382
Done:  383
Done:  384
Done:  385
Done:  386
Done:  387
Done:  388
Done:  389
Done:  390
No matching indices
Done:  391
Done:  392
Done:  393
No ma